# First PySpark Program Continued

## Overview

Our program is planned as follows:
We want to find the most used words in Pride and Prejudice. Here are the steps we want to take:
1. `Read` input data (assuming a plain text file)
2. `Token`ize each word
3. `Clean` up: 
   1. Remove puncuations and non-word tokens
   2. Lowercase each word
4. `Count` the frequency of each word
5. `Answer` return the top 10 (or 20, 50, 100)

In chapter 2 we've done 1~3. Now we want to do 4~5,submit our first PySpark program, and also organize our program into multiple Python files.

## Group, Order and Aggregate the Records

### Group

To group a data frame's records into groups, we use data frame's `groupby()` method and pass the columns we want to group as the parameter. 

This method returns a `GroupData` object on which we can apply aggregation functions such as `count()`.

In [19]:
# The usual initialization
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode, lower, regexp_extract, length, col

spark = SparkSession.builder.appName("Analyzing the vocabulary of Pride and Prejudice").getOrCreate()

# load and preprocess data
book = spark.read.text("../data/gutenberg_books/1342-0.txt")
words_nonull = book.select(
        split(book.value, " ").alias("line")
    ).select(
        explode("line").alias("word")
    ).select(
        lower("word").alias("word_lower")
    ).select(
        regexp_extract("word_lower", "[a-z]+", 0).
            alias("word")
    ).where(
        length(col("word")) > 0
    )

In [20]:
# group
groups = words_nonull.groupBy(col("word"))
print(groups)

In [82]:
results = words_nonull.groupBy(
        col("word")
    ).count()
results.show(5)


+------+-----+
|  word|count|
+------+-----+
|online|    4|
|  some|  209|
| still|   72|
|   few|   72|
|  hope|  122|
+------+-----+
only showing top 5 rows



### Order

Use `orderBy` to order the results by a column. You can pass multiple columns.

In [30]:
results.orderBy("count", ascending=False).show(5)
# alternativly
ordered_results = results.orderBy(col("count").desc())

+----+-----+
|word|count|
+----+-----+
| the| 4496|
|  to| 4235|
|  of| 3719|
| and| 3602|
| her| 2223|
+----+-----+
only showing top 5 rows



## Write data from a data frame

Just as we use `read()` and the `SparkReader` to read data in Spark, we use `write()` and the `SparkWriter` to write data from a data frame to disk. Let's do a CSV.

Note that Spark will not write to a single CSV file but create a folder with the path we specified and write as many output files as there are partitions and a `_SUCCESS` file. It doesn't guarantee any order unless you do a `orderBy` first. However, if your data is big, it is quite expensive to order them before writing to disk. Furthermore, since it is normally another distributed program that will read the written data, there's generally no point ordering the data at write time).



In [31]:
results.write.mode("overwrite").csv("./data/simple_count")
ordered_results.write.mode("overwrite").csv("./data/ordered_count")

To specify the desired number of partitions, apply `coalesce()` method.

In [32]:
ordered_results.coalesce(1).write.csv("./data/coalesced")

## Simplify your Code

### Simplify dependency import

Instead of importing each function we need in the `pyspark.sql.functions` module, import the whole module as an object `F`. (Don't import `*` from the package, otherweise your readers cannot tell which functions you use are from the package, which are Python native functions).

In [34]:
import pyspark.sql.functions as F

### Simplify with method chaining

Each data frame method returns the data frame after the transformation, so you can chain the transformations.

In [36]:
results = (
    spark.read.text("../data/gutenberg_books/1342-0.txt")
    .select(F.split(F.col("value"), " ").alias("line"))
    .select(F.explode(F.col("line")).alias("word"))
    .select(F.lower(F.col("word")).alias("word"))
    .select(F.regexp_extract(F.col("word"), "[a-z']*", 0).alias("word"))
    .where(F.length(F.col("word")) > 0)
    .groupBy("word")
    .count()
)

results.orderBy("count", ascending=False).show(5)

+----+-----+
|word|count|
+----+-----+
| the| 4480|
|  to| 4218|
|  of| 3711|
| and| 3504|
| her| 2199|
+----+-----+
only showing top 5 rows



## Submit to PySpark to Run as a Batch

The code is in `./code/word_count_submit.py`. We read all text files in the `../data/gutenberg_books/` folder and find the most frequent words.

Under the project folder, run

```bash
$ spark-submit ./03\ submit\ pyspark\ program/code/word_count_submit.py
```

We'll see a bunch of INFO and our result. The file is also proerly written to where we specified.



## Exercises

### Exercise 3.3 Challenge

Wrap your program in a function that takes a file name as a param- eter. It should return the number of distinct words.

===

The `spark-submit` accepts the following options

```bash
./bin/spark-submit \
  --class <main-class> \
  --master <master-url> \
  --deploy-mode <deploy-mode> \
  --conf <key>=<value> \
  ... # other options
  <application-jar> \
  [application-arguments]
```

What we pass `[[application-arguments]]` simply after our script file. (If we're not running locally, we must first specify the master URL in `--master` option before giving our script file).

Then in our script, the parameters will be in the `sys.argv` array, starting from index 1 as index 0 is the script file.

We wrote the file as accepting two parameters: the text file path and the top N frequent words to show.

```bash
$ spark-submit ./03\ submit\ pyspark\ program/code/parameterized.py \
    "./data/gutenberg_books/11-0.txt" \
    10                                         
```

### Exercise 3.4 

Modify the script to return a sample of five words that appear only once in Jane Austen’s Pride and Prejudice.

In [41]:
results.where(col("count") == 5).select(col("word")).show(5)

+----------+
|      word|
+----------+
| arguments|
| solemnity|
|likelihood|
|     parts|
|    absurd|
+----------+
only showing top 5 rows



### Exercise 3.5
1. Using the substring function (refer to PySpark’s API or the pyspark shell if needed), return the top five most popular first letters (keep only the first letter of each word).
2. Compute the number of words starting with a consonant or a vowel. (Hint: The isin() function might be useful.)

In [50]:
(
    results.select(
        F.substring(
            col("word"),1,1
        ).alias("first_letter")
    ).
    groupBy("first_letter")
    .count()
    .orderBy(col("count"), ascending=False)
    .show(5)
)

+------------+-----+
|first_letter|count|
+------------+-----+
|           s|  679|
|           c|  612|
|           p|  529|
|           a|  527|
|           d|  485|
+------------+-----+
only showing top 5 rows



In [73]:
first_letters = results.select(
    F.substring(col("word"), 1, 1).alias("first_letter")
).where(col("first_letter").rlike("[a-z]")) # make sure to only keep the words starting with a letter, not '

total_word_count = first_letters.count()

vowel_word_count = first_letters.where(
    col("first_letter").isin(["a", "e", "i", "o", "u"])
).count()

consonant_word_count = total_word_count - vowel_word_count

print(total_word_count, vowel_word_count, consonant_word_count)

6577 1619 4958


### Exercise 3.6

Why doesn't the following work?

```
my_data_frame.groupby("my_column").count().sum()
```

When using `count()` in aggregation, it called on the `GroupData` object returned by the `groupBy`, which returns two columns: the originally grouped column plus a column named `count`. `sum()` only works on one column, so it cannot be applied to the result of `count()`.